In [1]:
def do_all(line_iterators, map, get_hash, reduce):
    mapped_lines = []
    for line_iterator in line_iterators:
        for i, line in enumerate(line_iterator):
            # skip first line with csv headers
            if i == 0:
                continue
            line = line.replace('\n', '')
            mapped_lines.append(map(line))
    sorting_key = lambda x: get_hash(x)
    sorted_lines = sorted(mapped_lines, key = sorting_key)
    return reduce(sorted_lines)

# 1

In [7]:
def map(line):
    s = line.split(',')
    return s[0] + ',' + s[3]

def get_hash(line):
    return hash(line.split(',')[0])

def reduce_temperature(lines):
    values = []
    current_city = None
    result = []
    for line in lines:
        s = line.split(',')
        if current_city is None:
            current_city = s[0]
        if current_city == s[0]:
            values.append(float(s[1]))
        else:
            result.append(current_city + ',' + str(sum(values) / len(values)))
            current_city = s[0]
            values = [float(s[1])]
    result.append(current_city + ',' + str(sum(values) / len(values)))
    return result

with open('city_weather.csv', 'r') as f:
    avg_city = do_all([f], map, get_hash, reduce_temperature)
avg_city

['Воронеж,1.3451612903225807',
 'Орел,-0.20967741935483875',
 'Калуга,-0.8709677419354839',
 'Москва,-0.1516129032258064',
 'Краснодар,3.0',
 'Нижний Новгород,-1.6193548387096774',
 'Благовещенск,-10.993548387096775',
 'Хабаровск,-6.416129032258064',
 'Архангельск,-7.225806451612903',
 'Биробиджан,-9.251612903225807',
 'Кызыл,-14.812903225806451',
 'Тула,-0.24516129032258066',
 'Улан-Удэ,-12.093548387096773',
 'Кострома,-1.7774193548387098',
 'Курск,1.1290322580645162',
 'Тюмень,-3.487096774193548',
 'Иркутск,-10.141935483870967',
 'Нальчик,1.935483870967742',
 'Пенза,-0.9516129032258065',
 'Грозный,4.6',
 'Великий Новгород,-2.02258064516129',
 'Майкоп,3.774193548387097',
 'Вологда,-3.164516129032258',
 'Абакан,-7.161290322580645',
 'Владимир,-1.4',
 'Томск,-8.493548387096775',
 'Омск,-6.941935483870967',
 'Махачкала,5.129032258064516',
 'Иваново,-1.729032258064516',
 'Липецк,-0.964516129032258',
 'Салехард,-13.40967741935484',
 'Оренбург,-5.338709677419355',
 'Смоленск,-1.270967741935

# 2

In [ ]:
# к формату город,округ,температура
def map_merge_step(line):
    s = line.split(',')
    if len(s) == 2:
        return s[1] + ',' + s[0] + ','
    else:
        return s[0] + ',,' + s[3]

# по городу
def get_hash_merge_step(line):
    return line.split(',')[0]

def reduce_merge_step(lines):
    current_region = None
    current_city = None
    current_result_buffer = []
    result = []
    for line in lines:
        s = line.split(',')
        if current_city == None:
            current_city = s[0]
        if s[0] != current_city:
            current_result_buffer = []
            current_city = None
            current_region = None
            continue
        if s[1] != '':
            current_region = s[1]
            for buffered in current_result_buffer:
                buffered[1] = current_region
                result.append(','.join(buffered))
            continue
        if current_region == None:
            current_result_buffer.append(s)
        else:
            s[1] = current_region
            result.append(','.join(s))
    for buffered in current_result_buffer:
        buffered[1] = current_region
        result.append(','.join(buffered))
    current_result_buffer = []
    current_city = None
    current_region = None
    return result

with open('city_weather.csv', 'r') as cities:
    with open('district_to_city.csv', 'r') as districts:
        city_with_region = do_all([cities, districts], map_merge_step, get_hash_merge_step, reduce_merge_step)

city_with_region

def map_region_step(line):
    s = line.split(',')
    return s[1] + ',' + s[2]

def get_hash_region_step(line):
    return hash(line.split(',')[0])

# функция reduce подходит из первого задания
avg_region = do_all([city_with_region], map_region_step, get_hash_region_step, reduce_temperature)
avg_region